Sport City Index

NY, WAS, CHI, & LA

NY: Knicks, Mets (NYN), Giants, Rangers
WAS: Wizards, Nationals (WAS), Capitals, Commanders
CHI: Bulls, Cubs (CHN), Bears, Blackhawks
LA: Lackers, Dodgers (LAN), Rams, Kings

Win is +1
Loss is -1

In [71]:
import pandas as pd
import json

In [72]:
with open('/Users/alexfreeman/Documents/dev/scratch-sports-city-index/data/city_team_map.json') as f:
    city_map = json.load(f)

pd.DataFrame(city_map)

,city,mlb,nba,nhl,nfl
0,Washington DC,WAS,WAS,tbd,tbd
1,New York City,NYN,NYK,NYR,NYG
2,Los Angeles,LAL,LAL,tbd,tbd
3,Chicago,CHN,CHI,tbd,tbd


In [73]:
# for c in city_map:
#     print(c['mlb'])

[c['mlb'] for c in city_map]

['WAS', 'NYN', 'LAL', 'CHN']

In [74]:
# 1 row per game. need to double up so that each team gets a row per game
mlb_full = pd.read_csv('/Users/alexfreeman/Documents/dev/scratch-sports-city-index/data/MLB2020-2024GameInfo_small.csv')
mlb_full.head()

,Date,Game Number,Day of Week,VT,VT League,VT Game Number,HT,HT League,HT Game Number,VT Score,HT Score,Game Winner
0,20240320,0,Wed,LAN,NL,1,SDN,NL,1,5,2,1
1,20240321,0,Thu,SDN,NL,2,LAN,NL,2,15,11,1
2,20240328,0,Thu,COL,NL,1,ARI,NL,1,1,16,0
3,20240328,0,Thu,WAS,NL,1,CIN,NL,1,2,8,0
4,20240328,0,Thu,SLN,NL,1,LAN,NL,3,1,7,0


In [75]:
mlb_full['VT'].unique()

array(['LAN', 'SDN', 'COL', 'WAS', 'SLN', 'PIT', 'SFN', 'ANA', 'DET',
       'NYA', 'MIN', 'CLE', 'BOS', 'TOR', 'CHN', 'MIL', 'ATL', 'CIN',
       'KCA', 'TEX', 'MIA', 'CHA', 'ARI', 'NYN', 'TBA', 'SEA', 'BAL',
       'PHI', 'OAK', 'HOU'], dtype=object)

In [76]:
mlb_teams = [c['mlb'] for c in city_map]
mlb = mlb_full[
    mlb_full['VT'].isin(mlb_teams)
    |
    mlb_full['HT'].isin(mlb_teams)
]

mlb.head()

,Date,Game Number,Day of Week,VT,VT League,VT Game Number,HT,HT League,HT Game Number,VT Score,HT Score,Game Winner
3,20240328,0,Thu,WAS,NL,1,CIN,NL,1,2,8,0
14,20240328,0,Thu,CHN,NL,1,TEX,AL,1,3,4,0
18,20240329,0,Fri,MIL,NL,1,NYN,NL,1,3,1,1
26,20240330,0,Sat,WAS,NL,2,CIN,NL,2,7,6,1
29,20240330,0,Sat,MIL,NL,2,NYN,NL,2,7,6,1


In [77]:
mlb[mlb['Date'] == 20240330]

,Date,Game Number,Day of Week,VT,VT League,VT Game Number,HT,HT League,HT Game Number,VT Score,HT Score,Game Winner
26,20240330,0,Sat,WAS,NL,2,CIN,NL,2,7,6,1
29,20240330,0,Sat,MIL,NL,2,NYN,NL,2,7,6,1
39,20240330,0,Sat,CHN,NL,2,TEX,AL,2,2,11,0


In [100]:
mlb[(mlb['VT'] == 'WAS') | (mlb['HT'] == 'WAS')]

,Date,Game Number,Day of Week,VT,VT League,VT Game Number,HT,HT League,HT Game Number,VT Score,HT Score,Game Winner
3,20240328,0,Thu,WAS,NL,1,CIN,NL,1,2,8,0
26,20240330,0,Sat,WAS,NL,2,CIN,NL,2,7,6,1
41,20240331,0,Sun,WAS,NL,3,CIN,NL,3,5,6,0
62,20240401,0,Mon,PIT,NL,5,WAS,NL,4,8,4,1
89,20240403,0,Wed,PIT,NL,6,WAS,NL,5,3,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
10549,20200923,0,Wed,PHI,NL,57,WAS,NL,56,12,3,1
10561,20200924,0,Thu,NYN,NL,57,WAS,NL,57,3,2,1
10589,20200926,1,Sat,NYN,NL,58,WAS,NL,58,3,4,0
10590,20200926,2,Sat,NYN,NL,59,WAS,NL,59,3,5,0


In [92]:
# mlb_long = (
#     pd.concat([
#         mlb[mlb['VT'].isin(mlb_teams)]
#         , mlb[mlb['HT'].isin(mlb_teams)]
#         ]
#         )
#     [['Date', 'VT', 'HT', 'VT Score', 'HT Score', 'Game Winner']]
# )

# li = []
# for n in mlb_long['Game Winner']:
#     if n == 0:
#         li.append(-1)
#     else:
#         li.append(n)
# mlb_long['index_score'] = li
# mlb_long.head()

In [113]:
daily_scores = []

for _, row in mlb_full.iterrows():
# for _, row in mlb.iterrows():
    date = row['Date']
    vt_team = row['VT']
    ht_team = row['HT']
    winner = row['Game Winner']

    if winner == 1: # visiting team wins
        daily_scores.append({'date': str(date), 'team': vt_team, 'index_score': 1})
        daily_scores.append({'date': str(date), 'team': ht_team, 'index_score': -1})
    elif winner == 0: # home team wins
        daily_scores.append({'date': str(date), 'team': vt_team, 'index_score': -1})
        daily_scores.append({'date': str(date), 'team': ht_team, 'index_score': 1})

mlb_daily_index = pd.DataFrame(daily_scores)

mlb_daily_index['date'] = pd.to_datetime(mlb_daily_index['date'].astype(str), format='%Y%m%d')
mlb_daily_index['month'] = mlb_daily_index['date'].dt.to_period('M').dt.to_timestamp()
mlb_daily_index['league'] = 'mlb'

In [114]:
mlb_daily_index[mlb_daily_index['team' ] == 'WAS']

,date,team,index_score,month,league
6,2024-03-28,WAS,-1,2024-03-01,mlb
52,2024-03-30,WAS,1,2024-03-01,mlb
82,2024-03-31,WAS,-1,2024-03-01,mlb
125,2024-04-01,WAS,-1,2024-04-01,mlb
179,2024-04-03,WAS,1,2024-04-01,mlb
...,...,...,...,...,...
21099,2020-09-23,WAS,-1,2020-09-01,mlb
21123,2020-09-24,WAS,-1,2020-09-01,mlb
21179,2020-09-26,WAS,1,2020-09-01,mlb
21181,2020-09-26,WAS,1,2020-09-01,mlb


NBA

In [117]:
nba_full = pd.read_csv('/Users/alexfreeman/Documents/dev/scratch-sports-city-index/data/nba_regular_season_totals_2010_2024_small.csv')
nba_full.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL
0,2022-23,1610612744,GSW,Golden State Warriors,22201230,2023-04-09 0:00:00,GSW @ POR,W
1,2020-21,1610612749,MIL,Milwaukee Bucks,22000051,2020-12-29 0:00:00,MIL @ MIA,W
2,2013-14,1610612751,BKN,Brooklyn Nets,21300359,2013-12-16 0:00:00,BKN vs. PHI,W
3,2013-14,1610612757,POR,Portland Trail Blazers,21300347,2013-12-14 0:00:00,POR @ PHI,W
4,2018-19,1610612745,HOU,Houston Rockets,21801200,2019-04-07 0:00:00,HOU vs. PHX,W


In [118]:
nba_full[nba_full['GAME_ID'] == 22201230]

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL
0,2022-23,1610612744,GSW,Golden State Warriors,22201230,2023-04-09 0:00:00,GSW @ POR,W
27720,2022-23,1610612757,POR,Portland Trail Blazers,22201230,2023-04-09 0:00:00,POR vs. GSW,L


In [ ]:
nba_full['index_score'] = nba_full['WL'].map({'W': 1, 'L': -1})
nba = nba_full[['GAME_DATE', 'TEAM_ABBREVIATION', 'index_score']]
nba.columns = ['date', 'team',  'index_score']
nba['date'] = pd.to_datetime(nba['date']).dt.floor('D')
nba['month'] = nba['date'].dt.to_period('M').dt.to_timestamp()
nba['league'] = 'nba'
nba

/var/folders/f3/4nbhf7y135b8xgz1cqcb842w0000gn/T/ipykernel_2821/914520090.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba['date'] = pd.to_datetime(nba['date']).dt.floor('D')
/var/folders/f3/4nbhf7y135b8xgz1cqcb842w0000gn/T/ipykernel_2821/914520090.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba['month'] = nba['date'].dt.to_period('M').dt.to_timestamp()
/var/folders/f3/4nbhf7y135b8xgz1cqcb842w0000gn/T/ipykernel_2821/914520090.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

,team,date,index_score,month,league
0,GSW,2023-04-09,1,2023-04-01,nba
1,MIL,2020-12-29,1,2020-12-01,nba
2,BKN,2013-12-16,1,2013-12-01,nba
3,POR,2013-12-14,1,2013-12-01,nba
4,HOU,2019-04-07,1,2019-04-01,nba
...,...,...,...,...,...
33311,CHA,2023-01-23,-1,2023-01-01,nba
33312,LAC,2023-01-29,-1,2023-01-01,nba
33313,CHI,2019-03-23,-1,2019-03-01,nba
33314,BOS,2021-10-30,-1,2021-10-01,nba
